## checking sizes of alert postage stamps

looking at multi-resolution stamps

In [44]:
import sys
import io
from astropy.nddata import CCDData
import astropy.io.fits as fits
from lsst.alert.packet import retrieve_alerts

In [8]:
with open('/sdf/group/rubin/user/ebellm/workspace/cosmos/alerts/11830050.avro','rb') as f:
    writer_schema, alert_iter = retrieve_alerts(f)

In [9]:
record = next(alert_iter)

In [11]:
record['cutoutDifference']

b'SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                  -32 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                   33                                                  NAXIS2  =                   33                                                  EXTEND  =                    T                                                  CUTMINX =                   21                                                  CUTMINY =                  -10                                                  BUNIT   = \'nJy     \'                                                            WCSAXES =                    2 / Number of coordinate axes                      CRPIX1  =      16.571157624175 / Pixel coordinate of reference point            CRPIX2  =      16.647434020941 / Pixel coordinate of reference point            PC1_1   = -7.6767969571084E-08 / Coo

In [13]:
sys.getsizeof(record['cutoutDifference'])

17313

In [14]:
sys.getsizeof(record['cutoutTemplate'])

17313

In [20]:
with open('test_diff_stamp_from_alert.fits','wb') as f:
    f.write(record['cutoutDifference'])

In [21]:
!ls -l test_diff_stamp_from_alert.fits

-rw-r--r-- 1 ebellm rubin_users 17280 Sep 15 23:52 test_diff_stamp_from_alert.fits


In [33]:
!gzip test_diff_stamp_from_alert.fits
!ls -l test_diff_stamp_from_alert.fits.gz

-rw-r--r-- 1 ebellm rubin_users 6586 Sep 15 23:52 test_diff_stamp_from_alert.fits.gz


not identical but very close

In [17]:
ccd = CCDData.read(io.BytesIO(record['cutoutDifference']))

In [18]:
ccd

CCDData([[        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         ...,
         [-5.2575154 , -2.5556386 ,  4.1538324 , ...,  1.8321458 ,
           4.2387714 , -1.6243646 ],
         [ 1.6422311 ,  0.4236933 ,  5.1306324 , ...,  2.8470976 ,
           2.140074  ,  1.6280992 ],
         [-2.5434291 , -7.913455  , -0.5198002 , ...,  0.06647457,
          -1.3805475 , -0.33206978]], unit='nJy')

In [64]:
dir(ccd)

['__abstractmethods__',
 '__array__',
 '__array_prepare__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_arithmetic',
 '_arithmetic_data',
 '_arithmetic_mask',
 '_arithmetic_meta',
 '_arithmetic_uncertainty',
 '_arithmetic_wcs',
 '_data',
 '_flags',
 '_handle_wcs_slicing_error',
 '_insert_in_metadata_fits_safe',
 '_mask',
 '_meta',
 '_prepare_then_do_arithmetic',
 '_psf',
 '_slice',
 '_slice_mask',
 '_slice_uncertainty',
 '_slice_wcs',
 '_uncertainty',
 '_unit',
 '_wcs',
 'add',
 'convert_unit_to',
 'copy',
 'data',
 'divide',
 'dtype',
 'flags',
 'header',
 'known_invalid_fits_unit_strings',
 'mask',
 'max',
 'mean',

In [22]:
ccd.write('test_diff_stamp_from_ccddata.fits')

In [23]:
!ls -l test_diff_stamp_from_ccddata.fits

-rw-r--r-- 1 ebellm rubin_users 17280 Sep 15 23:55 test_diff_stamp_from_ccddata.fits


now let's synthesize a MEF 

In [26]:
hdulist = ccd.to_hdu()

In [29]:
hdulist[1]

In [31]:
hdulist[1].data.shape

(33, 33)

In [36]:
# alerce computation of number of required pixels, omitting centers
def alerce_multiscale_npixels(ncenter,nlevels):
    return  ncenter * (1 + (nlevels-1) * 0.75)

In [37]:
alerce_multiscale_npixels(15**2., 5)

900.0

for MEF though we can't remove the central pixels, as this assumes.

In [38]:
5*15**2.

1125.0

In [39]:
33**2.

1089.0

In [52]:
1125/900

1.25

In [76]:
fake_data_hdu = hdulist[0].copy()
fake_variance_hdu = hdulist[1].copy()

In [77]:
fake_data = hdulist[0].data[:15,:15]
fake_variance = hdulist[1].data[:15,:15]

In [78]:
fake_data.shape

(15, 15)

In [79]:
fake_data_hdu.data = fake_data
fake_variance_hdu.data = fake_variance

In [80]:
type(hdulist)

astropy.io.fits.hdu.hdulist.HDUList

In [85]:
multiscale_hdulist = fits.hdu.hdulist.HDUList()
#del multiscale_hdulist[0]
#del multiscale_hdulist[1]

In [86]:
multiscale_hdulist

[]

In [87]:
for i in range(5):
    multiscale_hdulist.append(fake_data_hdu)
    multiscale_hdulist.append(fake_variance_hdu)

In [88]:
multiscale_hdulist

[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7fdbb4205190>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7fdbb44ee910>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7fdbb4de3990>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7fdbb44ee910>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7fdbb709d590>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7fdbb44ee910>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7fdbb6afee10>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7fdbb44ee910>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7fdbb48d8dd0>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7fdbb44ee910>]

In [89]:
multiscale_hdulist[3].shape

(15, 15)

In [90]:
multiscale_hdulist.writeto('test_n5_15x15_multiscale_stamp.fits')

In [91]:
!ls -l test_n5_15x15_multiscale_stamp.fits

-rw-r--r-- 1 ebellm rubin_users 57600 Sep 18 04:59 test_n5_15x15_multiscale_stamp.fits


In [93]:
!rm -f test_n5_15x15_multiscale_stamp.fits.gz
!gzip test_n5_15x15_multiscale_stamp.fits
!ls -l test_n5_15x15_multiscale_stamp.fits.gz

-rw-r--r-- 1 ebellm rubin_users 1829 Sep 18 04:59 test_n5_15x15_multiscale_stamp.fits.gz


In [63]:
!pwd

/sdf/data/rubin/user/ebellm/workspace/ap_pipe-notebooks/notebooks
